In [1]:
import deepcut
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow import keras

In [2]:
x_train = ['วันนี้วันจันทร์','วันนี้วันศุกร์','วันนี้วันเสาร์','วันนีวันจันทนะ','วันนีวันศุกนะ','วันนีวันศุร์นะ']
y_train = ['ถูก','ถูก','ถูก','ผิด','ผิด','ผิด']
x_test = ['วันนี้วันพุธ','วันนี้วันอังคาร','วันนี้วันอังคาน',]

In [3]:
test = deepcut.tokenize('วันนี้วันศุร์นะ')
print(test)

['วัน', 'นี้', 'วัน', 'ศุร์นะ']


In [4]:
def word_tokenize(corpus):
    x = 0
    words = []
    for i in corpus:
        list_word = deepcut.tokenize(i,custom_dict='custom_dict.txt') #ตัดคำ
        words.append(list_word)
        x += 1
    return words

words = word_tokenize(x_train)
print(words)
print(len(words))

[['วัน', 'นี้', 'วัน', 'จันทร์'], ['วัน', 'นี้', 'วัน', 'ศุกร์'], ['วัน', 'นี้', 'วัน', 'เสาร์'], ['วัน', 'นีวัน', 'จันทนะ'], ['วัน', 'นีวัน', 'ศุก', 'นะ'], ['วัน', 'นีวัน', 'ศุร์นะ']]
6


In [5]:
words1 = []
x = 0
y = 0
for text in range(len(words)):
    for word in range(len(words[y])):
        words1.append(words[y][x])
        x += 1
    y += 1
    x = 0
words1 = set(words1)

vocab = []
for list_d in words1:
    vocab.append(list_d)
vocab = [' '] + vocab
print(vocab)
print(len(vocab))

[' ', 'นะ', 'เสาร์', 'วัน', 'จันทนะ', 'นี้', 'ศุกร์', 'ศุก', 'นีวัน', 'จันทร์', 'ศุร์นะ']
11


In [6]:
def sent2idx(x):
    xidx = []
    for w in x:
#         print(w)
        if w in vocab:
            xidx.append(vocab.index(w))
    return np.array(xidx)

In [7]:
sent2idx(words[0])

array([3, 5, 3, 9])

In [14]:
def word_seq(words):
    Xtrain_idx = []
    maxlen = 0
    for x in words:
        Xtrain_idx.append(sent2idx(x))
        if len(Xtrain_idx[-1]) > maxlen:
            maxlen = len(Xtrain_idx[-1])
#         print(len(Xtrain_idx[-1]))
    return Xtrain_idx

In [15]:
Xtrain = word_seq(words)
print(Xtrain)

[array([3, 5, 3, 9]), array([3, 5, 3, 6]), array([3, 5, 3, 2]), array([3, 8, 4]), array([3, 8, 7, 1]), array([ 3,  8, 10])]


In [17]:
Xtrain = sequence.pad_sequences(np.array(Xtrain), maxlen=5,padding='pre')
print(Xtrain)

[[ 0  3  5  3  9]
 [ 0  3  5  3  6]
 [ 0  3  5  3  2]
 [ 0  0  3  8  4]
 [ 0  3  8  7  1]
 [ 0  0  3  8 10]]


In [18]:
Xtrain.shape

(6, 5)

In [19]:
vocab_size = len(vocab)+1

In [27]:
model = keras.Sequential()
model.add(keras.layers.Embedding(input_dim = vocab_size,output_dim = 300,input_length=5))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(8, return_sequences=True)))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(4)))
model.add(keras.layers.Dense(32, activation=keras.activations.relu))
model.add(keras.layers.Dense(2))

In [28]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer=keras.optimizers.SGD(learning_rate=1e-4))

In [29]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 5, 300)            3600      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 5, 16)             19776     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 8)                 672       
_________________________________________________________________
dense_16 (Dense)             (None, 32)                288       
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 66        
Total params: 24,402
Trainable params: 24,402
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
model.fit(Xtrain, np.array([0,0,0,1,1,1]), epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 2ms/step - loss: 2.1096
Epoch 2/10
1/1 [==============================] - 0s 1000us/step - loss: 6.1182
Epoch 3/10
1/1 [==============================] - 0s 1000us/step - loss: 5.9935
Epoch 4/10
1/1 [==============================] - 0s 1ms/step - loss: 5.7675
Epoch 5/10
1/1 [==============================] - 0s 2ms/step - loss: 3.5221
Epoch 6/10
1/1 [==============================] - 0s 1000us/step - loss: 0.6931
Epoch 7/10
1/1 [==============================] - 0s 2ms/step - loss: 0.6931
Epoch 8/10
1/1 [==============================] - 0s 1ms/step - loss: 0.6931
Epoch 9/10
1/1 [==============================] - 0s 1ms/step - loss: 0.6931
Epoch 10/10
1/1 [==============================] - 0s 2ms/step - loss: 0.6931


In [31]:
model.predict(Xtrain).argmax(axis=1)

array([0, 0, 0, 0, 0, 1], dtype=int64)